**Loading Data**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/gdrive/MyDrive/handsigns.zip','r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
!rm -rf /content/output

**Creating CNN**

In [ ]:
!pip -q install keras-tuner

In [ ]:
import keras
import os
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from kerastuner import RandomSearch

In [ ]:
def Model(hp):
  cnn = Sequential()
  cnn.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                 input_shape=(150, 150, 3), 
                 activation='relu')) 
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
   
  cnn.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                 activation='relu'))
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
  
  cnn.add(Flatten())

  cnn.add(Dense(units=hp.Int('dense_1_units', 
                min_value=64, max_value=512, step=16),
                activation='relu'))
  cnn.add(Dropout(rate=0.5))

  cnn.add(Dense(units=4, activation='softmax'))

  cnn.compile(optimizer=keras.optimizers.RMSprop(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return cnn

In [ ]:
tuner = RandomSearch(Model,
                    objective='val_accuracy',
                    max_trials=5,
                    directory='/content/outputs',
                    project_name="Hands Gestures Detection")

**Preprocessing of Images**

In [ ]:
def preprocessing(dir_path):
  train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.1)
  
  training_set = train_datagen.flow_from_directory(
        dir_path,
        target_size=(150, 150),
        batch_size=10,
        class_mode='categorical',
        subset='training')
  
  validation_generator = train_datagen.flow_from_directory(
      dir_path,
      target_size=(150, 150),
      batch_size=10,
      class_mode='categorical',
      subset='validation')
  
  return (training_set, validation_generator)

In [ ]:
data = preprocessing('/content/datasets/train')

Found 4918 images belonging to 4 classes.
Found 545 images belonging to 4 classes.


In [ ]:
tuner.search(data[0],
                    validation_data = data[1],
                    epochs=4)

Trial 5 Complete [00h 02m 50s]
val_accuracy: 0.9724770784378052

Best val_accuracy So Far: 0.9944953918457031
Total elapsed time: 00h 13m 43s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 70, 70, 96)        153696    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 96)        0         
_________________________________________________________________
flatten (Flatten)            (None, 117600)            0         
_________________________________________________________________
dense (Dense)                (None, 384)               45158784  
_________________________________________________________________
dropout (Dropout)            (None, 384)               0

In [ ]:
model.fit(data[0],
          epochs=10,
          initial_epoch=3,
          validation_data=data[1])

Epoch 4/10
492/492 [==============================] - 40s 82ms/step - loss: 0.2560 - accuracy: 0.9437 - val_loss: 0.0427 - val_accuracy: 0.9890
Epoch 5/10
492/492 [==============================] - 40s 82ms/step - loss: 0.6986 - accuracy: 0.9516 - val_loss: 0.0189 - val_accuracy: 0.9945
Epoch 6/10
492/492 [==============================] - 40s 82ms/step - loss: 0.1722 - accuracy: 0.9599 - val_loss: 0.0094 - val_accuracy: 0.9963
Epoch 7/10
492/492 [==============================] - 40s 82ms/step - loss: 0.3890 - accuracy: 0.9579 - val_loss: 0.0423 - val_accuracy: 0.9908
Epoch 8/10
492/492 [==============================] - 40s 82ms/step - loss: 0.3268 - accuracy: 0.9614 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 9/10
492/492 [==============================] - 40s 82ms/step - loss: 0.3028 - accuracy: 0.9573 - val_loss: 0.0806 - val_accuracy: 0.9706
Epoch 10/10
492/492 [==============================] - 40s 82ms/step - loss: 0.2320 - accuracy: 0.9669 - val_loss: 0.0132 - val_accuracy

In [ ]:
model.save("/gdrive/MyDrive/handsigns.h5")

In [ ]:
data[0].class_indices

{'0Fingers': 0, '1Fingers': 1, '2Fingers': 2, '5Fingers': 3}

In [ ]:
CNN = keras.models.load_model("/gdrive/MyDrive/handsigns.h5")

In [ ]:
# Prediction for Image using our trained model
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/content/1348_150_150.jpg", target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = CNN.predict(test_image)

In [ ]:
if int(result[0][0]) == 1:
  print("0Fingers")
elif int(result[0][1]) == 1:
  print("1Fingers")
elif int(result[0][2]) == 1:
  print("2Fingers")
elif int(result[0][3]) == 1:
  print("5Fingers")

2Fingers
